## **Extracción y Preparación Preliminar de los Datos de Fallas**

El presente dataset fue construido con datos de las bases de datos de diferentes sistemas de la Empresa de Energía, los cuales fueron suministrados directamente por XYZ, información recopilada del sistema OMS (Outage Management System) y del GIS (Geographic Information System) según el filtro por municipio correspondiente, pues el Dataset contiene información en Años, Meses, Días, Horas, Minutos y Segudos de las fallas o aperturas que se han presentado en el periodo comprendido desde el 01/01/2021 hasta el 30/06/2023 [Dos (2) años y medio (1/2)] de las aperturas presentadas en los municipios de: **PIJAO**, **BUENAVISTA**, **MONTENEGRO**, **CALARCÁ**, y **SALENTO** del Departamento del Quindío.

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

import os

# Cambia el directorio actual al de la carpeta en Google Drive
os.chdir("/content/drive/MyDrive/EAM/Especializacion_Analitica_de_Datos/Data_XYZ")

# Lista el contenido de la carpeta
#!ls

Mounted at /content/drive


Cargamos el Dataset: "*DATASET_FALLAS.csv*" el cual contiene la información de las fallas y alimentado con el campo del código del elemento relacionado a la falla (Sin considerar la ZONA (Rural: **R** - URBANO: **U**, así como tampoco con el MUNICIPIO) de las fallas.

In [2]:
df_fallas = pd.read_csv('DATASET_FALLAS.csv')
df_fallas = df_fallas.drop(['DURACION'], axis=1)
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22


Procedemos a cargar el Dataset: "***DATASET_TRANSFORMADORES_-_T_MUNICIPIO_ZONA.csv***" el cual contiene la relación Código-Reconectador, su Municipio y su respectiva zona, y usando left Join (Unión por Izquierda) con los códigos de los fallas relacionados como equipos en las aperturas, y se procede a empalmar con el data inicial y así logrando complementar el Dataset con las zonas de los Transformadores:

In [3]:
#----------------------------- JOIN CON TRANSFORMADORES -------------------------
df_t = pd.read_csv('DATASET_TRANSFORMADORES_-_T_MUNICIPIO_ZONA.csv')
df_t = df_t.rename(columns={'NODO_TRANSFORM_V':'CODIGO','MUNICIPIO':'MUNICIPIO_T'}) #Se aplica rename de las columas para poder hacer Left Join (Con el codigo) y para un mejor entendimiento del proceso (Con la zona).
df_t.head()

,CODIGO,MUNICIPIO_T,ZONA_T
0,ARUP0287,SALENTO,U
1,ARUP0287,SALENTO,U
2,ARUQ0288,SALENTO,R
3,BAUN0045,CALARCÁ,U
4,BAUN0046,CALARCÁ,U


In [4]:
df_fallas = df_fallas.merge(df_t, on='CODIGO', how='left')
#df_fallas['ZONA_T'] = data['ZONA_T'].fillna(data['ZONA_R'])
df_fallas

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN
...,...,...,...,...,...
44403,30/06/2023 20:41:46,30/06/2023 20:51:08,R-098,NaN,NaN
44404,30/06/2023 20:41:54,30/06/2023 20:52:45,R-098,NaN,NaN
44405,30/06/2023 20:47:10,01/07/2023 12:17:20,QURQ0380,NaN,NaN
44406,30/06/2023 20:48:37,02/07/2023 07:48:42,CARQ0405,CALARCÁ,R


Procedemos a cargar el Dataset: "***DATASET_RECONECTADORES_-_R__MUNICIPIO_ZONA.csv***" el cual contiene la relación Código-Reconectador, su municipio y su respectiva zona, y usando left Join (Unión por Izquierda) con los códigos de los RECONECTADORES relacionados como equipos en las aperturas se procede a empalmar con el data inicial y así logrando complementar el Dataset que ya tenía las zonas de los Transformadores; ahora con las de los Reconectadores:

In [5]:
#----------------------------- JOIN CON RECONECTADORES -------------------------
data_r = pd.read_csv('DATASET_RECONECTADORES_-_R_MUNICIPIO_ZONA.csv')
data_r = data_r.rename(columns={'MUNICIPIO':'MUNICIPIO_R'})
data_r.head()

,CODIGO,MUNICIPIO_R,ZONA_R
0,R-005,CALARCÁ,R
1,R-006,CALARCÁ,U
2,R-007,CALARCÁ,U
3,R-008,MONTENEGRO,U
4,R-009,MONTENEGRO,U


In [6]:
df_fallas = df_fallas.merge(data_r, on='CODIGO', how='left')
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN


In [7]:
df_fallas['MUNICIPIO_T_R'] = df_fallas['MUNICIPIO_T'].fillna(df_fallas['MUNICIPIO_R'])
df_fallas['ZONA_T_R'] = df_fallas['ZONA_T'].fillna(df_fallas['ZONA_R'])
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN


Procedemos a cargar el Dataset: "***DATASET_AISLADEROS_-_A__MUNICIPIO_ZONA.csv***" el cual (Similar que el dataset de "DATASET_RECONECTADORES_ZONA.csv" ) contiene la relación Código-Aislador, su municipio y su respectiva zona, y usando left Join (Unión por Izquierda) con los códigos de los AISLADEROS relacionados como equipos en las aperturas; se procede a empalmar con el data Aacumulada y así logrando complementar el Dataset que ya tenía las zonas de los Transformadores y de los Reconectadores; ahora con las de los Aisladeros:

In [8]:
#----------------------------- JOIN CON AISLADEROS -------------------------
data_a = pd.read_csv('DATASET_AISLADEROS_-_A_MUNICIPIO_ZONA.csv')
data_a = data_a.rename(columns={'MUNICIPIO':'MUNICIPIO_A'})
data_a.head()

,CODIGO,MUNICIPIO_A,ZONA_A
0,SPR-060,CALARCÁ,U
1,S-PR18,MONTENEGRO,R
2,S-PR22,CALARCÁ,U
3,S-PR39,CALARCÁ,R
4,S-PR40,CALARCÁ,R


In [9]:
df_fallas = df_fallas.merge(data_a, on='CODIGO', how='left')
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,ZONA_A
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_fallas['MUNICIPIO_T_R_A'] = df_fallas['MUNICIPIO_T_R'].fillna(df_fallas['MUNICIPIO_A'])
df_fallas['ZONA_T_R_A'] = df_fallas['ZONA_T_R'].fillna(df_fallas['ZONA_A'])
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,ZONA_A,MUNICIPIO_T_R_A,ZONA_T_R_A
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Procedemos a cargar el Dataset: "***DATASET_CUCHILLAS_-_C__MUNICIPIO_ZONA.csv***" el cual (Similar que el dataset de "DATASET_RECONECTADORES_ZONA.csv" y que el de: "DATASET_AISLADEROS_ZONA.csv" ) contiene la relación Código-Cuchilla/Switch, su municipio y su respectiva zona correspondiente, y usando left Join (Unión por Izquierda) con los códigos de las CUCHILLAS/SWITCHES relacionados como equipos en las aperturas; se procede a empalmar con el data Acumulada y así logrando complementar el Dataset que ya tenía las zonas de los Transformadores, de los Reconectadores y de los Aisladeros; ahora con las de las Cuchillas/Switches:

In [11]:
#------------------------------- JOIN CON CUCHILLAS ---------------------------
data_c = pd.read_csv('DATASET_CUCHILLAS_-_C_MUNICIPIO_ZONA.csv')
data_c = data_c.rename(columns={'MUNICIPIO':'MUNICIPIO_C'})
data_c.head()

,CODIGO,MUNICIPIO_C,ZONA_C
0,C-0002,CALARCÁ,U
1,C-0004,MONTENEGRO,U
2,C-0004,MONTENEGRO,R
3,C-0005,MONTENEGRO,U
4,C-0006,MONTENEGRO,U


In [12]:
df_fallas = df_fallas.merge(data_c, on='CODIGO', how='left')
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,ZONA_A,MUNICIPIO_T_R_A,ZONA_T_R_A,MUNICIPIO_C,ZONA_C
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_fallas['MUNICIPIO_T_R_A_C'] = df_fallas['MUNICIPIO_T_R_A'].fillna(df_fallas['MUNICIPIO_C'])
df_fallas['ZONA_T_R_A_C'] = df_fallas['ZONA_T_R_A'].fillna(df_fallas['ZONA_C'])
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,ZONA_A,MUNICIPIO_T_R_A,ZONA_T_R_A,MUNICIPIO_C,ZONA_C,MUNICIPIO_T_R_A_C,ZONA_T_R_A_C
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Procedemos a cargar el Dataset: "***DATASET_CIRCUITOS_-_F__MUNICIPIO_ZONA.csv.csv***" el cual contiene la relación Código-Circuito, su municipio y su respectiva zona correspondiente, y usando left Join (Unión por Izquierda) con los códigos de los circuitos relacionados como equipos en las aperturas; se procede a empalmar con el data Acumulada y así logrando complementar el Dataset que ya tenía las zonas de los Transformadores, de los Reconectadores y de los Aisladeros y de las cichillas; ahora con las de los circuitos:

In [14]:
#------------------------------- JOIN CON CIRCUITOS (FEEDERS) ---------------------------
data_f = pd.read_csv('DATASET_CIRCUITOS_-_F_MUNICIPIO_ZONA.csv')
data_f = data_f.rename(columns={'MUNICIPIO':'MUNICIPIO_F'})
data_f.head()

,CODIGO,MUNICIPIO_F,ZONA_F
0,206-22-,BUENAVISTA,R
1,206-23-,BUENAVISTA,R
2,206-24-,BUENAVISTA,R
3,206-25-,BUENAVISTA,R
4,206-31,BUENAVISTA,R


In [15]:
df_fallas = df_fallas.merge(data_f, on='CODIGO', how='left')
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,ZONA_A,MUNICIPIO_T_R_A,ZONA_T_R_A,MUNICIPIO_C,ZONA_C,MUNICIPIO_T_R_A_C,ZONA_T_R_A_C,MUNICIPIO_F,ZONA_F
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_fallas['MUNICIPIO_T_R_A_C_F'] = df_fallas['MUNICIPIO_T_R_A_C'].fillna(df_fallas['MUNICIPIO_F'])
df_fallas['ZONA_T_R_A_C_F'] = df_fallas['ZONA_T_R_A_C'].fillna(df_fallas['ZONA_F'])
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,CODIGO,MUNICIPIO_T,ZONA_T,MUNICIPIO_R,ZONA_R,MUNICIPIO_T_R,ZONA_T_R,MUNICIPIO_A,...,MUNICIPIO_T_R_A,ZONA_T_R_A,MUNICIPIO_C,ZONA_C,MUNICIPIO_T_R_A_C,ZONA_T_R_A_C,MUNICIPIO_F,ZONA_F,MUNICIPIO_T_R_A_C_F,ZONA_T_R_A_C_F
0,01/01/2021 00:00:01,04/01/2021 12:48:33,ARUQ1435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,QUUQ0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,205-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Eliminación de variables Irrelevantes:**
Se deciden borrar las siguientes variables por no aportar información relevante al caso de estudio:

- **ZONA_T_R_A_C**
- **ZONA_T_R_A**
- **ZONA_T_R**
- **ZONA_T**
- **ZONA_R**
- **ZONA_A**
- **ZONA_C**
- **ZONA_F**

- **MUNICIPIO_T_R_A_C**
- **MUNICIPIO_T_R_A**
- **MUNICIPIO_T_R**
- **MUNICIPIO_T**
- **MUNICIPIO_R**
- **MUNICIPIO_A**
- **MUNICIPIO_C**
- **MUNICIPIO_F**

- **CODIGO**

Toda vez que las ZONAS a lo largo del proceso de Extracción se fueron acumulando en las mismas (Columnas temporales) finalmente quedando la columna: **"ZONA_T_R_A_C_F"** la cual contiene el valor de ZONA definitivo para los Transformadores, Reconectadores, Aisladeros, Cuchillas/Switches y circuitos, acto seguido se reemplaza el nombre de la misma por el de **"ZONA"**, y que los MUNICIPIOS a lo largo del proceso de Extracción se fueron acumulando en las mismas (Columnas temporales) finalmente quedando la columna: **"MUNICIPIO_T_R_A_C_F"** la cual contiene el valor de ZONA definitivo para los Transformadores, Reconectadores, Aisladeros, Cuchillas/Switches y cirvuitos, acto seguido se reemplaza el nombre de la misma por el de **"MUNICIPIO"**.

In [17]:
df_fallas = df_fallas.rename(columns={'MUNICIPIO_T_R_A_C_F': 'MUNICIPIO','ZONA_T_R_A_C_F': 'ZONA'})
df_fallas = df_fallas.drop(['MUNICIPIO_T_R_A_C','MUNICIPIO_T_R_A', 'MUNICIPIO_T_R','MUNICIPIO_T','MUNICIPIO_R','MUNICIPIO_A','MUNICIPIO_C','MUNICIPIO_F',
                           'ZONA_T_R_A_C','ZONA_T_R_A', 'ZONA_T_R','ZONA_T','ZONA_R','ZONA_A','ZONA_C','ZONA_F',
                            'CODIGO'], axis=1)

df_fallas.head()

,FECHA_INICIO,FECHA_FIN,MUNICIPIO,ZONA
0,01/01/2021 00:00:01,04/01/2021 12:48:33,NaN,NaN
1,01/01/2021 00:00:01,11/01/2021 11:42:17,NaN,NaN
2,01/01/2021 00:02:28,01/01/2021 05:19:21,NaN,NaN
3,01/01/2021 00:02:33,01/01/2021 05:19:34,NaN,NaN
4,01/01/2021 00:02:48,01/01/2021 05:19:47,NaN,NaN


In [18]:
df_fallas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45072 entries, 0 to 45071
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FECHA_INICIO  45072 non-null  object
 1   FECHA_FIN     45072 non-null  object
 2   MUNICIPIO     14955 non-null  object
 3   ZONA          14883 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


Realizamos una depuración del dataset de fallas de manera tal que eliminamos los registros relacionados los municipios NO considerados en el caos de estudio como los serían: Armenia, Filandia, Circasia, Quimbaya, Tebaida, Cordoba y Genova del Departamento del Quindío:

In [19]:
df_fallas = df_fallas.dropna(subset=['MUNICIPIO'])

In [20]:
df_fallas.head()

,FECHA_INICIO,FECHA_FIN,MUNICIPIO,ZONA
10,01/01/2021 10:31:12,01/01/2021 10:54:02,SALENTO,U
12,01/01/2021 10:36:56,01/01/2021 15:39:02,PIJAO,R
18,01/01/2021 15:14:42,09/01/2021 14:22:15,SALENTO,R
22,01/01/2021 16:13:49,01/01/2021 18:16:02,SALENTO,R
23,01/01/2021 16:21:03,01/01/2021 18:15:38,SALENTO,R


In [21]:
df_fallas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14955 entries, 10 to 45071
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FECHA_INICIO  14955 non-null  object
 1   FECHA_FIN     14955 non-null  object
 2   MUNICIPIO     14955 non-null  object
 3   ZONA          14883 non-null  object
dtypes: object(4)
memory usage: 584.2+ KB


Finalmente tenemos el Dataset preliminar, el cual se encuentra con la información necesaria para empezar a realiar su respectiva transformación y preparación para el proceso de Analítica en cuanto a las aperturas (Fallas) se refiere.

Se procede a almacenar el dataset preliminar con el nombre: **"DATASET_FALLAS_-_2021-01-01_-_2023-06-30_-_V5"** en formato **xlsx** quedando listo para su consumo y uso con un total de 14.955 registros:

In [22]:
# Cambiamos el directorio actual al de la carpeta en Google Drive
os.chdir("/content/drive/MyDrive/EAM/Especializacion_Analitica_de_Datos/Proyecto_de_Grado/Datasets/DS_Base")

# Lista el contenido de la carpeta
!ls

DATASET_FALLAS_-_2021-01-01_-_2023-06-30_-_V5.xlsx  Temperatura.csv
Precipitacion.csv				    Velocidad_Viento.csv


In [23]:
#df_fallas.to_excel('DATASET_FALLAS_-_2021-01-01_-_2023-06-30_-_V5.xlsx', index=False)
df_fallas.to_excel('DATASET_FALLAS_-_BASE.xlsx', index=False)